# Valuing Options Using Multi-Period Binomial Tree Model
### Shantanu Laghate

### Hacker Hour - Financial Programming (Part 1)

### Feb 15, 2019

Model Calibrated using Amazon Data from Jan 1st 2015 - Jan 1st 2018 using the Black-Scholes assumption of constant returns

In [ ]:
import quandl
import matplotlib as pyplot
import pandas as pd
import numpy as np

with open("key.txt") as key:
    quandl.ApiConfig.api_key = key.read()

## Downloading Amazon Data 

Closing stock price data from Jan 1 2015 - Jan 1 2018 is used.

In [ ]:
data = quandl.get("WIKI/AMZN", start_date="2015-01-01", end_date="2018-01-01")
data.head()

In [ ]:
data["Close"].plot()
# During this 3-year period, the Amazon stock almost tripled in price. This has a marked impact on our model, as shown below.

## Calculating Percentage Returns

Returns are calculated by through the formula:

$\frac{\Delta S}{S} = \frac{S_{i}}{S_{i-1}} - 1$

After this is generated, the expected daily expected return is simply the mean of this array, and the daily volatility is the standard deviation of this array. In this case, $\Delta T = 1$.

To find the monthly expected returns, $\Delta T$ increases to 21 trading days per month. For yearly, $\Delta T = 252$. The daily expected returns are simply multiplied by $\Delta T$ to obtain these values.

For volatility over a longer period of time, the daily value must be multiplied by $\sqrt{\Delta T}$. 

In [ ]:
close = data["Close"]
returns = (close/close.shift(1)) - 1
returns.plot()

In [ ]:

print("Daily Expected Returns: ", returns.mean())
print("Monthly Expected Returns: ", returns.mean()*21)
print("Yearly Expected Returns: ", returns.mean()*252)

print()

print("Daily Volatility: ", returns.std())
print("Monthly Volatility: ", returns.std()*np.sqrt(21))
print("Yearly Volatility: ", returns.std()*np.sqrt(252))

## Creating the Multi-period Binomial Model

To calibrate a binomial model, we must first specify a time step. For this first model, let's use a daily time step, $\Delta T = 1$. Let's also assume that $r = 1$, which is somewhat unrealistic since the 1-year risk free rate increased from 0.25% to 1.8% during the indicated time period, but this will simplify calculations.

$u = e^{\sigma \sqrt{\Delta T}}$ and $d = e^{-\sigma \sqrt{\Delta T}}$. These values are first calculated.

Then we can calculate the risk-neutral probability $p = \frac{e^{r \Delta T} - d}{u - d}$

The starting price of the model is assumed to be today's (11/8/18) price: \$ 1754.91

In [ ]:
u = np.exp(returns.std())
d = np.exp(-returns.std())
print("u = ", u)
print("d = ", d)

In [ ]:
p = (np.exp(1/252) - d)/(u - d)
p

In [ ]:
n = len(close)
tree = np.zeros((n, n))
tree[0][0] = 1636.39
for i in range(1, n):
    tree[0][i] = tree[0][i-1]*u

for i in range(1, n):
    for j in range(i, n):
        tree[i][j] = tree[i-1][j-1]*d
print(tree[:3, -3:])

## Options Pricing with the Binomial Model


In [ ]:
# Call Option
k = 2100
t = 30
actual_price = 4.90

def call(s, k):
    return max(0, s - k)


otree = np.zeros((t+1, t+1))
for i in range(t+1):
    otree[i][t] = call(tree[i][t], k)

    
#print(tree[:6, :6])



for i in range(t-1, -1, -1):
    for j in range(i+1):
        otree[j][i] = max(p*otree[j][i+1] + (1-p)*otree[j+1][i+1], call(tree[j][i], k))
#print(otree)
print(otree[0][0])

In [ ]:
# Put Option
k = 1720
t = 162
actual_price = 4.90

def put(s, k):
    return max(0, k - s)

np.set_printoptions(precision=2)
otree = np.zeros((t+1, t+1))
for i in range(t+1):
    otree[i][t] = put(tree[i][t], k)
#print(tree[:6, :6])
for i in range(t-1, -1, -1):
    for j in range(i+1):
        otree[j][i] = max(p*otree[j][i+1] + (1-p)*otree[j+1][i+1], 0)
#print(otree)
print(otree[0][0])

### Calls 
Amazon Calls:

| Expiry | Strike | Market Price | Model Prediction |
|:------:|:-:|:-:|:-:|
| 11/16/18 | 1700.00 | 56.80 | 100.96 |
| 12/21/18 | 2100.00 | 4.90 | 30.57 |
| 6/22/19  | 1720.00  |  220.00 | 1808.55  |

Theoretically, there is no benefit of early exercise of an american call option, so the model predicts the same price for both european and american calls.

### Puts
Amazon Puts:

| Expiry | Strike | Market Price | Model Prediction (Euro) | Model Prediction (American) |
|:------:|:-:|:-:|:-:|:-:|
| 11/16/18 | 1700.00 | 15.54 | 3.77 | 4.53 | 
| 12/21/18 | 2100.00 | 430.83 | 153.80 | 345.08 |
| 6/22/19  | 1720.00  |  155.36 | 0.27 | 15.87 |


As you can see, the model prediction is widely off the market price. There is a tendency to massively over-value calls, and massively under-value puts. One reason for this is because the risk neutral probability $p = 0.6$, which means that there is a higher likelihood of the stock price increasing. The model is influenced very heavily by the strong upward movement of the stock price during 2015-2018. As mentioned above, the price almost tripled.

This sways calls to be priced much higher in the model since there is a belief that the price of the stock will go higher, and puts to be priced lower, since the price of the stock will not go down by much.
